### this section is for code notes, not important to the notebook

In [1]:
#import requests, json, datetime, pandas as pd

#unix_date_time = (datetime.datetime.today().strftime("%s"))

#response_manchester = requests.get("http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=42.9956&lon=-71.4548&dt="+ unix_date_time +"&appid=fd3ef9b217de69f13fde1bd0325da289")
#processed_responce = response_manchester.json()

#df = pd.DataFrame(processed_responce['hourly'])
#df
#df2 = df[["dt","temp", "humidity", "wind_speed"]].copy()
#df.to_csv(r'/home/anthony/Documents/programing-materals/comp574/WetherApiProject/export_dataframe.csv', index = False, header=True)



#df2

#dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'current', 'hourly'])


## This section starts us off (api query and processing) 
* This section loops through 5 dates and gets the hrly temp, humidity and wind_speed for 5 days

* then it process it into a single large dataframe for processing later

In [2]:
'''
The new goal of my project is to: calculate the average temp in manchester over the last 5 days and graph it
'''
# unix time calculator -0> https://www.vultr.com/resources/unix-time-calculator/?convert=date&date_day=27&date_month=10&date_year=2021&date_hour=12&date_minute=00&date_second=00&date_tz_from=UTC&date_tz_to=UTC&date_format=Y-m-d+H%3Ai%3As

import requests, json, datetime, pandas as pd

df_final = pd.DataFrame(columns=['dt', 'temp', 'humidity', 'wind_speed']) # inititalisation of final dataframe
last_5_days = []

for x in range(0,5): # gets current date and time delta in unixtime
    unix_date_time = (datetime.datetime.today() - datetime.timedelta(x)).strftime("%s") 
    last_5_days.append(unix_date_time)

for day in last_5_days: # gets the json from the api, processes it into a df and appends it to the final frame. ownly takes 4 columbs
    response_manchester = requests.get("http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=42.9956&lon=-71.4548&dt="+ day +"&appid=fd3ef9b217de69f13fde1bd0325da289")
    
    processed_responce = response_manchester.json() # gets json
    
    df_raw = pd.DataFrame(processed_responce['hourly']) # json -> df from the "hourly" list
    
    df_processed = df_raw[["dt","temp", "humidity", "wind_speed"]].copy() # cuts df down to what we want ownly
    
    df_final = df_final.append(df_processed) # appends to the final df


## This section checks the df.shape

 * This section just makes shure we have the right number of columbs and rows

 * hould have (119, 4)

In [3]:
df_final.shape

(98, 4)

## this sends df_final to a csv file
* thats it, its just a csv file

In [4]:
df_final.to_csv(r'/home/anthony/Documents/programing-materals/comp574/DataProject/export_dataframe.csv', index = False, header=True)

#### converts the temp into ferinheit and  dt into a readable date in place of a unix time stamp

In [5]:
df_final.loc[:, "temp"] = df_final["temp"].apply(lambda x: x - 273)
df_final.loc[:, "dt"] = df_final["dt"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%m/%d/%Y'))
df_final

#datetime.datetime.utcfromtimestamp(posix_time).strftime('%Y-%m-%dT%H:%M:%SZ')

,dt,temp,humidity,wind_speed
0,11/14/2021,6.58,90,2.68
1,11/14/2021,4.36,90,2.68
0,11/13/2021,12.32,96,0.00
1,11/13/2021,12.23,96,0.89
2,11/13/2021,11.05,95,1.34
...,...,...,...,...
19,11/10/2021,14.27,63,1.79
20,11/10/2021,13.29,63,1.34
21,11/10/2021,12.00,56,1.34
22,11/10/2021,10.55,56,0.45


## This sectoin handles averaging the columbs

#### Avearge Temp

In [6]:
aveage_temp = df_final['temp'].mean()
print(aveage_temp) # this needs to be converted to regular temp

8.830714285714285


#### Average humidity

In [7]:
average_humidity = df_final['humidity'].mean()
print(average_humidity)

78.21428571428571


#### Average wind speed

In [8]:
aveage_wind_speed = df_final['wind_speed'].mean()
print(aveage_wind_speed) # This is in MPH already, its just so slow that its under 1 mph

0.8740816326530614


In [9]:
# get the date and time of the last 5 days, convert to unixtime

#for eatch unix date get the wether for manchester on that day, append that data to the df

# take the average temp